In [3]:
import pandas as pd
import numpy as np
import os
import cv2
import natsort
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob

# data 1 : URFD
- step 1 : pack each directory containing .png files as .avi file (to follow UCF format)
- step 2 : re-arange video files according to the class label
- step 3 : copy-and-paste UCF clip generation code...

In [4]:
root = '/data/FallDownData/URFD/raw'
video_files = natsort.natsorted(glob.glob(root + '/*/*'))
for video in tqdm(video_files):
    prefix, ext = os.path.splitext(video)
    save_path = prefix.replace('raw', 'video') + '.avi'
    
    frames = []
    
    cap = cv2.VideoCapture(video)
    frame_height, frame_width = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    out = cv2.VideoWriter(save_path,
                          cv2.VideoWriter_fourcc('M','J','P','G'), 
                          30, (frame_width//2,frame_height))
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # left : depth, right : color
        color_frame = frame[:, frame_width//2:]
        out.write(color_frame)

    os.system('mkdir -p {}'.format(os.path.dirname(save_path)))
    out.release()

100%|██████████| 100/100 [00:32<00:00,  4.55it/s]


# data 2 : Multicam FDD
- this dataset is bullshit!
- syncronization is not matched well

In [ ]:
root = '/data/FallDownData/MulticamFD/raw'
df = pd.read_csv(os.path.join(os.path.dirname(root), 'Multicam_Annotations.csv'))

In [ ]:
df.head()

In [ ]:
def arange_multicam_dataset(root):
    videos = []

    for dirpath, dirnames, filenames in os.walk(root):
        for fn in filenames:
            name, ext = os.path.splitext(fn)
            if ext == '.avi':
                videos.append(os.path.join(dirpath, fn))


    videos = natsort.natsorted(videos)
    scenarios = np.unique([os.path.dirname(p) for p in videos])
    
    for scenario in tqdm(scenarios):
        def get_shortest_frame_count():

            shortest_frame_count = np.inf

            for i in range(1,9):
                vpath = os.path.join(scenario, f'cam{i}.avi')
                cap = cv2.VideoCapture(vpath)
                frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
                if frame_count < shortest_frame_count:
                    shortest_frame_count = frame_count

            shortest_frame_count = int(shortest_frame_count)

            return shortest_frame_count

        shortest_frame_count = get_shortest_frame_count()

        def get_syncronized_videos():
            res = []
            for i in range(1,9):
                vpath = os.path.join(scenario, f'cam{i}.avi')
                cap = cv2.VideoCapture(vpath)
                frames = []
                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    frames.append(frame)

                res.append(frames[-shortest_frame_count:])

            return np.array(res)

        syncronized = get_syncronized_videos()

        scenario_id = eval(os.path.basename(scenario).split('chute')[1].lstrip('0'))
        scenario_video_home = os.path.dirname(scenario).replace('raw', 'video')

        for seg_ix, (start, end, code) in enumerate(df[df.id==scenario_id][["start","end","code"]].values):
            segment = syncronized[:, start:end]  # a frame segment

            label = 'fall' if code == 2 else 'adl'
            scenario_video = os.path.join(scenario_video_home, label)

            os.system('mkdir -p {}'.format(scenario_video))

            for i in range(1,9):
                seg_ix_str = str(seg_ix+1).zfill(2)
                vpath = os.path.join(scenario, f'cam{i}.avi')
                scenario_indicator = os.path.basename(scenario)

                seg_vpath = os.path.join(scenario_video, f'{scenario_indicator}-cam{i}-s{seg_ix_str}.avi')

                frame_height, frame_width = segment[i-1][0].shape[:2]

                fps = cv2.VideoCapture(vpath).get(cv2.CAP_PROP_FPS)

                out = cv2.VideoWriter(seg_vpath,
                                  cv2.VideoWriter_fourcc('M','J','P','G'), 
                                  fps, (frame_width,frame_height))

                for frame in segment[i-1]:
                    out.write(frame)

                    
arange_multicam_dataset(root)

In [ ]:
cap = cv2.VideoCapture('/data/FallDownData/MulticamFD/video/fall/chute09-cam6-s05.avi')

frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

def check_detection_result(img, box):
    #bx,by,bw,bh = 307.466186523,196.785614014,86.6007080078,166.478393555
    bx,by,bw,bh = box
    xmin, ymin, xmax, ymax = int(bx-bw/2), int(by-bh/2), int(bx+bw/2), int(by+bh/2)
    return img[ymin:ymax, xmin:xmax]

plt.imshow(check_detection_result(frames[16], (370.211120605,174.12197876,88.2487030029,67.7461776733)))

In [ ]:
plt.imshow(frames[17])

In [ ]:
cap.get(cv2.CAP_PROP_FPS)

In [ ]:
with open('/data/torch_data/UCF-101/ucfTrainTestlist/trainlist01.txt') as f:
    ucfList = f.readlines()
    labels = [ line.strip().split()[1] for line in ucfList ]

In [ ]:
np.unique(labels, return_counts=True)

# Create Traintests-plit for cross-validation

In [5]:
from sklearn.model_selection import KFold

In [6]:
def create_traintestList(root, annotation_path, random_state=0, n_splits=5):
    os.system(f'mkdir -p {annotation_path}')

    video_dirs = natsort.natsorted(glob.glob(root + '/*/*'))

    np.random.seed(random_state)
    np.random.shuffle(video_dirs)

    class2idx = {'adl':0, 'fall':1}

    formated_video_dirs = np.array([ x[len(root.rstrip('/'))+1:] for x in video_dirs ])

    kf = KFold(n_splits)

    for k, (train_ix, test_ix) in enumerate(kf.split(formated_video_dirs)):
        _train, _test = np.array(formated_video_dirs)[train_ix], np.array(formated_video_dirs)[test_ix]
        _train_lab = [ os.path.dirname(x) for x in _train ]
        _test_lab = [ os.path.dirname(x) for x in _test ]
        print(f'[splist-{k}] train : {np.unique(_train_lab, return_counts=True)}, test : {np.unique(_test_lab, return_counts=True)}')
        for _split, _data in zip(['train', 'test'], [_train, _test]):
            lines = []
            for i in range(len(_data)):
                line = _data[i] + " "
                if _split == 'train':
                    line += str(class2idx[os.path.dirname(_data[i])])

                lines.append(line + '\n')

            with open(os.path.join(annotation_path, f"{_split}list{k+1:02d}.txt"), 'w') as fp:
                fp.writelines(lines)

In [7]:
# URFD
create_traintestList(root = '/data/FallDownData/URFD/video/',
                     annotation_path = '/data/FallDownData/URFD/TrainTestlist',
                     n_splits=5)

[splist-0] train : (array(['adl', 'fall'], dtype='<U4'), array([29, 49])), test : (array(['adl', 'fall'], dtype='<U4'), array([10, 10]))
[splist-1] train : (array(['adl', 'fall'], dtype='<U4'), array([35, 43])), test : (array(['adl', 'fall'], dtype='<U4'), array([ 4, 16]))
[splist-2] train : (array(['adl', 'fall'], dtype='<U4'), array([27, 51])), test : (array(['adl', 'fall'], dtype='<U4'), array([12,  8]))
[splist-3] train : (array(['adl', 'fall'], dtype='<U4'), array([32, 47])), test : (array(['adl', 'fall'], dtype='<U4'), array([ 7, 12]))
[splist-4] train : (array(['adl', 'fall'], dtype='<U4'), array([33, 46])), test : (array(['adl', 'fall'], dtype='<U4'), array([ 6, 13]))


In [8]:
# MulticamFD
create_traintestList(root = '/data/FallDownData/MulticamFD/video/',
                     annotation_path = '/data/FallDownData/MulticamFD/TrainTestlist',
                     n_splits=5)

[splist-0] train : (array(['adl', 'fall'], dtype='<U4'), array([590, 140])), test : (array(['adl', 'fall'], dtype='<U4'), array([138,  45]))
[splist-1] train : (array(['adl', 'fall'], dtype='<U4'), array([578, 152])), test : (array(['adl', 'fall'], dtype='<U4'), array([150,  33]))
[splist-2] train : (array(['adl', 'fall'], dtype='<U4'), array([578, 152])), test : (array(['adl', 'fall'], dtype='<U4'), array([150,  33]))
[splist-3] train : (array(['adl', 'fall'], dtype='<U4'), array([588, 143])), test : (array(['adl', 'fall'], dtype='<U4'), array([140,  42]))
[splist-4] train : (array(['adl', 'fall'], dtype='<U4'), array([578, 153])), test : (array(['adl', 'fall'], dtype='<U4'), array([150,  32]))


In [ ]:
!ls /home/hossay/gaitanalysis/preprocess/data/

In [ ]:
path = '/data/FallDownData/URFD/raw/fall/fall-27-cam1-rgb/fall-27-cam1-rgb-060.png'
im = cv2.imread(path,-1)
plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))

In [ ]:
path = '/data/torch_data/fall-27-cam1.mp4'
cap = cv2.VideoCapture(path)
frames = []
while True:
    ret, frame = cap.read()
    if not ret: break
    frames.append(frame)

im = frames[60][:, 320:]
plt.figure(figsize=(15,15))
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))

cv2.imwrite('/home/hossay/Downloads/test.png', im)

In [ ]:
im.shape